# Solveur de Jacobi : Modèles de programmation Multi-GPU
Ce notebook présente 12 versions progressives d’un solveur de Jacobi 2D. Chaque section explique le modèle ou l’optimisation, compile la version, exécute et collecte les métriques Nsight.

## etape1_cpu
**Description :** Solveur Jacobi CPU de base : implémentation mono-thread. Utile pour valider la correction et les petites tailles de problème ; met en évidence la limite de calcul CPU.

**Intérêt :** Baseline : évalue la limite CPU pour établir une référence.

In [24]:
%%bash
cd etape1_cpu
make all

gcc -O2 -o main main.c


In [25]:
%%bash 
cd etape1_cpu
nsys profile -t openacc --stats=true --force-overwrite true -o main ./main

Terminé etape1_cpu
CPU time: 18.183735 seconds
Generating '/tmp/nsys-report-9a69.qdstrm'
[1/7] [========================100%] main.nsys-repepep
[2/7] [========================100%] main.sqlite2/7] [4%                          ] main.sqlite


SKIPPED: /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA trace data.
SKIPPED: /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA kernel data.
SKIPPED: /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain GPU memory data.
SKIPPED: /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain GPU memory data.
SKIPPED: /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain OpenACC event data.


[3/7] Executing 'cuda_api_sum' stats report
[4/7] Executing 'cuda_gpu_kern_sum' stats report
[5/7] Executing 'cuda_gpu_mem_time_sum' stats report
[6/7] Executing 'cuda_gpu_mem_size_sum' stats report
[7/7] Executing 'openacc_sum' stats report
Generated:
    /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.nsys-rep
    /home/corentin/Documents/Cours/M2/CHPS0904/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite


## etape2_cpu_gpu
**Description :** 1 CPU + 1 GPU + 1 stream : le CPU pilote le GPU via un unique stream CUDA. Le calcul Jacobi est entièrement délégué au GPU, le CPU ne fait que l’orchestration.

**Intérêt :** Met en évidence l'écart de performance CPU vs GPU lorsque la grille est suffisamment grande, dans un contexte réaliste d’utilisation d’un seul GPU et d’un seul stream.

### Compilation et exécution (1 CPU + 1 GPU + 1 stream)

In [7]:
%%bash 
cd etape2_cpu_gpu
make all

nvcc -O2 -o app main.cu kernel.cu


nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
 be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [8]:
%%bash 
cd etape2_cpu_gpu
nsys profile -t cuda --stats=true --force-overwrite true -o main ./app

Terminé etape2_cpu_gpu (1CPU + 1GPU + 1stream)
GPU time: 1.904882 seconds
Generating '/tmp/nsys-report-be73.qdstrm'
[1/6] [0%                          ] main.nsys-repCollecting data...
Generating '/tmp/nsys-report-be73.qdstrm'
[1/6] [========================100%] main.nsys-repepepepep
[1/6] [========================100%] main.nsys-rep
[2/6] [========================100%] main.sqlite2/6] [4%                          ] main.sqlite2/6] [7%                          ] main.sqlite2/6] [10%                         ] main.sqlite2/6] [14%                         ] main.sqlite2/6] [==20%                       ] main.sqlite2/6] [===24%                      ] main.sqlite6] [====26%                     ] main.sqlite2/6] [=====29%                    ] main.sqlite6] [=====31%                    ] main.sqlite [======33%                   ] main.sqlite2/6] [=======36%                  ] main.sqlite6] [=======38%                  ] main.sqlite [========40%                 ] main.sqlite2/6] [=========44%

## etape3_mpi_gpus
**Description :** MPI + GPUs : domaine réparti sur plusieurs rangs MPI, chacun pilotant un GPU. Illustrations des défis de mise à l'échelle multi-nœuds et des communications inter-rangs.

**Intérêt :** Test de montée en charge inter-nœuds et coût MPI sur cluster multi-GPU.

In [ ]:
%%bash 
cd etape3_mpi_gpus
make all

In [ ]:
%%bash 
cd etape3_mpi_gpus
nv-nsight-cu-cli --csv --report-file rapport_etape3_mpi_gpus.csv ./main
cat rapport_etape3_mpi_gpus.csv

## etape4_mpi_overlap
**Description :** MPI + recouvrement : recouvrements des échanges d’halo non-bloquants avec le calcul Jacobi local. Réduit l'impact de la latence réseau.

**Intérêt :** Cache la latence réseau en recouvrant communication et calcul local.

In [ ]:
%%bash 
cd etape4_mpi_overlap
make all

In [ ]:
%%bash 
cd etape4_mpi_overlap
nv-nsight-cu-cli --csv --report-file rapport_etape4_mpi_overlap.csv ./main
cat rapport_etape4_mpi_overlap.csv

## etape5_nccl
**Description :** NCCL : utilisation de la NVIDIA Collective Communications Library pour les échanges GPU à GPU. Montre les gains via NVLink ou PCIe haute bande passante.

**Intérêt :** Exploite automatiquement le topologie NVLink/PCIe pour des échanges GPU efficaces.

In [ ]:
%%bash
cd etape5_nccl
make all

In [ ]:
%%bash 
cd etape5_nccl
nv-nsight-cu-cli --csv --report-file rapport_etape5_nccl.csv ./main
cat rapport_etape5_nccl.csv

## etape6_nccl_overlap
**Description :** NCCL + recouvrement : superposition des collectifs NCCL avec le calcul sur GPU, cachant le coût de communication.

**Intérêt :** Essentiel à forte densité GPU pour maintenir les cœurs occupés.

In [ ]:
%%bash 
cd etape6_nccl_overlap
make all

In [ ]:
%%bash 
cd etape6_nccl_overlap 
nv-nsight-cu-cli --csv --report-file rapport_etape6_nccl_overlap.csv ./main
cat rapport_etape6_nccl_overlap.csv

## etape7_nccl_graphs
**Description :** NCCL + CUDA Graphs : capture et relecture des séquences Jacobi/échange pour réduire le surcoût des lancements.

**Intérêt :** Réduit l’overhead de lancement grâce aux CUDA Graphs.

In [ ]:
%%bash 
cd etape7_nccl_graphs
make all

In [ ]:
%%bash 
cd etape7_nccl_graphs
nv-nsight-cu-cli --csv --report-file rapport_etape7_nccl_graphs.csv ./main
cat rapport_etape7_nccl_graphs.csv

## etape8_nvshmem
**Description :** NVSHMEM : modèle PGAS à accès mémoire unilatéral GPU, simplifiant les mises à jour d’halo.

**Intérêt :** Simplifie les échanges via modèle PGAS unilatéral.

In [ ]:
%%bash 
cd etape8_nvshmem
make all

In [ ]:
%%bash 
cd etape8_nvshmem
nv-nsight-cu-cli --csv --report-file rapport_etape8_nvshmem.csv ./main
cat rapport_etape8_nvshmem.csv

## etape9_nvshmem_lt
**Description :** NVSHMEM + LTO : ajout de l’optimisation link-time pour inliner les fonctions critiques et réduire le coût des appels.

**Intérêt :** Optimisation link-time pour inliner les sections critiques.

In [ ]:
%%bash cd etape9_nvshmem_lt
make all

In [ ]:
%%bash 
cd etape9_nvshmem_lt
nv-nsight-cu-cli --csv --report-file rapport_etape9_nvshmem_lt.csv ./main
cat rapport_etape9_nvshmem_lt.csv

## etape10_vshmem_neighborhood_lto
**Description :** vshmem neighborhood_sync + LTO : synchronisation fine-grain de voisinage et optimisations link-time O2.

**Intérêt :** Synchronisation fine et LTO pour boucles serrées.

In [ ]:
%%bash 
cd etape10_vshmem_neighborhood_lto
make all

In [ ]:
%%bash 
cd etape10_vshmem_neighborhood_lto
nv-nsight-cu-cli --csv --report-file rapport_etape10_vshmem_neighborhood_lto.csv ./main 
cat rapport_etape10_vshmem_neighborhood_lto.csv

## etape11_nvshmem_norm_overlap_neighborhood_sync_lto
**Description :** Combinaison : NVSHMEM avec recouvrement, synchrone de voisinage, et LTO pour maximiser la concurrence.

**Intérêt :** Combinaison des meilleures pratiques pour un binaire ultra-optimisé.

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto
make all

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto
nv-nsight-cu-cli --csv --report-file rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv ./main
cat rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv

## etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1
**Description :** Tuning étendu : paramètres ajustables (taille de tuile, ordre de boucles) et hooks de benchmark.

**Intérêt :** Ajout de paramètres de tuning et hooks de benchmarking.

In [ ]:
%%bash 
cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 
make all

In [ ]:
%%bash 
cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 
nv-nsight-cu-cli --csv --report-file rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv ./main
cat rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv